In [1]:
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
parser = argparse.ArgumentParser(description='PyTorch actor-critic example')
parser.add_argument('--gamma', type=float, default=0.99, metavar='G',
                    help='discount factor (default: 0.99)')
parser.add_argument('--seed', type=int, default=543, metavar='N',
                    help='random seed (default: 543)')
parser.add_argument('--render', action='store_true',
                    help='render the environment')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='interval between training status logs (default: 10)')

_StoreAction(option_strings=['--log-interval'], dest='log_interval', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, help='interval between training status logs (default: 10)', metavar='N')

In [3]:
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--gamma G] [--seed N] [--render] [--log-interval N]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\25791\AppData\Roaming\jupyter\runtime\kernel-5c4ed7bf-8909-412b-81cb-a9807ba27791.json


SystemExit: 2

C:\Users\25791\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
class Arg():
    def __init__(self, gamma, seed, render, log_interval):
        self.gamma = gamma
        self.seed = seed
        self.render = render
        self.log_interval = log_interval
args = Arg(0.99, 42, 'store_true', 10)

In [5]:
# Cart Pole
env = gym.make('CartPole-v0')
env.seed(42)
torch.manual_seed(42)


SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])


class Policy(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)

        # actor's layer
        self.action_head = nn.Linear(128, 2)

        # critic's layer
        self.value_head = nn.Linear(128, 1)

        # action & reward buffer
        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        """
        forward of both actor and critic
        """
        x = F.relu(self.affine1(x))

        # actor: choses action to take from state s_t 
        # by returning probability of each action
        action_prob = F.softmax(self.action_head(x), dim=-1)

        # critic: evaluates being in the state s_t
        state_values = self.value_head(x)

        # return values for both actor and critic as a tuple of 2 values:
        # 1. a list with the probability of each action over the action space
        # 2. the value from state s_t 
        return action_prob, state_values


model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()


def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)

    # create a categorical distribution over the list of probabilities of actions
    m = Categorical(probs)

    # and sample an action using the distribution
    action = m.sample()

    # save to action buffer
    x = m.log_prob(action)
    print(probs)
    print(x)
    model.saved_actions.append(SavedAction(x, state_value))

    # the action to take (left or right)
    return action.item()


def finish_episode():
    """
    Training code. Calculates actor and critic loss and performs backprop.
    """
    R = 0
    saved_actions = model.saved_actions
    policy_losses = [] # list to save actor (policy) loss
    value_losses = [] # list to save critic (value) loss
    returns = [] # list to save the true values

    # calculate the true value using rewards returned from the environment
    for r in model.rewards[::-1]:
        # calculate the discounted value
        R = r + args.gamma * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()

        # calculate actor (policy) loss 
        policy_losses.append(-log_prob * advantage)

        # calculate critic (value) loss using L1 smooth loss
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))

    # reset gradients
    optimizer.zero_grad()

    # sum up all the values of policy_losses and value_losses
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

    # perform backprop
    loss.backward()
    optimizer.step()

    # reset rewards and action buffer
    del model.rewards[:]
    del model.saved_actions[:]


def main(iters):
    running_reward = 10

    # run inifinitely many episodes
    for i_episode in range(iters):

        # reset environment and episode reward
        state = env.reset()
        ep_reward = 0

        # for each episode, only run 9999 steps so that we don't 
        # infinite loop while learning
        for t in range(1, 10000):

            # select action from policy
            action = select_action(state)

            # take the action
            state, reward, done, _ = env.step(action)

            if args.render:
                env.render()

            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break

        # update cumulative reward
        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward

        # perform backprop
        finish_episode()

        # log results
        if i_episode % args.log_interval == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))

        # check if we have "solved" the cart pole problem
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break

In [6]:
main(1)

tensor([0.4513, 0.5487], grad_fn=<SoftmaxBackward>)
tensor([0.4265, 0.5735], grad_fn=<SoftmaxBackward>)
tensor([0.3996, 0.6004], grad_fn=<SoftmaxBackward>)
tensor([0.4277, 0.5723], grad_fn=<SoftmaxBackward>)
tensor([0.4012, 0.5988], grad_fn=<SoftmaxBackward>)
tensor([0.3712, 0.6288], grad_fn=<SoftmaxBackward>)
tensor([0.3454, 0.6546], grad_fn=<SoftmaxBackward>)
tensor([0.3209, 0.6791], grad_fn=<SoftmaxBackward>)
tensor([0.3432, 0.6568], grad_fn=<SoftmaxBackward>)
tensor([0.3185, 0.6815], grad_fn=<SoftmaxBackward>)
tensor([0.3398, 0.6602], grad_fn=<SoftmaxBackward>)
tensor([0.3127, 0.6873], grad_fn=<SoftmaxBackward>)
tensor([0.3333, 0.6667], grad_fn=<SoftmaxBackward>)
tensor([0.3049, 0.6951], grad_fn=<SoftmaxBackward>)
Episode 0	Last reward: 14.00	Average reward: 10.20
